# Linear Interpolation and Backoff (LIB) Classification

The LIB algorithm classifies URLs into genres by decomposing character n-grams and combining their probabilities using linear interpolation and backoff. It operates within a naive Bayes framework, where probabilities of n-grams (sequences of n characters) are used to estimate the likelihood of a URL belonging to a particular genre.

See: https://apps.dtic.mil/sti/pdfs/ADA599843.pdf


## Training Phase

The training phase involves:

1. Collecting a corpus of URLs and their associated genres.
2. Extracting n-grams from these URLs.
3. Calculating the frequency of each n-gram for each genre.
4. Estimating the interpolation coefficients (λ) using methods like information gain.
5. Storing these frequencies and coefficients to be used during the prediction phase.


In [1]:
import numpy as np
from collections import defaultdict, Counter
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import normalize

In [2]:
################################# Extract n-grams from a URL #################################

def extract_ngrams(url, n):
    """Extracts n-grams of length n from the given URL."""
    return [url[i:i+n] for i in range(len(url) - n + 1)]

################################# Calculate frequencies of n-grams #################################

def calculate_ngram_frequencies(urls, genres, n_range):
    """Calculates the frequency of n-grams for each genre from the given URLs and genres."""
    
    ngram_freq = defaultdict(lambda: defaultdict(int))
    genre_counts = Counter(genres)
    
    for url, genre in zip(urls, genres):
        for n in n_range:
            ngrams = extract_ngrams(url, n)
            for ngram in ngrams:
                ngram_freq[genre][ngram] += 1
    
    return ngram_freq, genre_counts

################################# Calculate the mutual information for n-grams #################################

def calculate_mutual_information(ngram_freq, genre_counts):
    """Calculates mutual information for n-grams using their frequencies and genre counts."""
    
    ngrams = list({ngram for genre in ngram_freq for ngram in ngram_freq[genre]})
    X = np.zeros((len(ngram_freq), len(ngrams)))
    y = list(genre_counts.keys())

    for i, genre in enumerate(y):
        for j, ngram in enumerate(ngrams):
            X[i, j] = ngram_freq[genre][ngram]
    
    mi = mutual_info_classif(X, y, discrete_features=True)
    return dict(zip(ngrams, mi))

################################# Estimate the interpolation coefficients #################################

def estimate_interpolation_coefficients(ngram_mi, n_range):
    """Estimates interpolation coefficients based on mutual information of n-grams."""
    
    lambda_sum = sum(ngram_mi.values())
    lambdas = {n: 0 for n in n_range}
    
    for ngram, mi in ngram_mi.items():
        n = len(ngram)
        lambdas[n] += mi / lambda_sum
    
    # Normalize the coefficients
    total = sum(lambdas.values())
    for n in lambdas:
        lambdas[n] /= total
    
    return lambdas

################################# Main training function #################################

def train_genre_classifier(urls, genres, n_range=(2, 3, 4)):
    """
    Trains the genre classifier by calculating n-gram frequencies, mutual information,
    and interpolation coefficients from the given URLs and genres.
    """
    # Calculate n-gram frequencies and genre counts
    ngram_freq, genre_counts = calculate_ngram_frequencies(urls, genres, n_range)
    
    # Calculate mutual information for n-grams
    ngram_mi = calculate_mutual_information(ngram_freq, genre_counts)
    
    # Estimate interpolation coefficients
    lambdas = estimate_interpolation_coefficients(ngram_mi, n_range)
    
    return ngram_freq, genre_counts, lambdas

In [3]:
# Example usage:
urls = ["www.exampleblog.com/post", "www.examplewiki.org/page", "shop.example.com/product"]
genres = ["blog", "wiki", "e-shop"]
n_range = (2, 3, 4)

ngram_freq, genre_counts, lambdas = train_genre_classifier(urls, genres, n_range)

print("N-gram Frequencies:", ngram_freq)
print("Genre Counts:", genre_counts)
print("Interpolation Coefficients:", lambdas)

N-gram Frequencies: defaultdict(<function calculate_ngram_frequencies.<locals>.<lambda> at 0x7fc8fbb3a290>, {'blog': defaultdict(<class 'int'>, {'ww': 2, 'w.': 1, '.e': 1, 'ex': 1, 'xa': 1, 'am': 1, 'mp': 1, 'pl': 1, 'le': 1, 'eb': 1, 'bl': 1, 'lo': 1, 'og': 1, 'g.': 1, '.c': 1, 'co': 1, 'om': 1, 'm/': 1, '/p': 1, 'po': 1, 'os': 1, 'st': 1, 'www': 1, 'ww.': 1, 'w.e': 1, '.ex': 1, 'exa': 1, 'xam': 1, 'amp': 1, 'mpl': 1, 'ple': 1, 'leb': 1, 'ebl': 1, 'blo': 1, 'log': 1, 'og.': 1, 'g.c': 1, '.co': 1, 'com': 1, 'om/': 1, 'm/p': 1, '/po': 1, 'pos': 1, 'ost': 1, 'www.': 1, 'ww.e': 1, 'w.ex': 1, '.exa': 1, 'exam': 1, 'xamp': 1, 'ampl': 1, 'mple': 1, 'pleb': 1, 'lebl': 1, 'eblo': 1, 'blog': 1, 'log.': 1, 'og.c': 1, 'g.co': 1, '.com': 1, 'com/': 1, 'om/p': 1, 'm/po': 1, '/pos': 1, 'post': 1, 'i.o': 0, 'lewi': 0, 'rg': 0, 'pro': 0, 'e.c': 0, 'wik': 0, 'ki.': 0, 'prod': 0, 'i.': 0, '.org': 0, 'rod': 0, 'pa': 0, 'wiki': 0, 'shop': 0, 'ge': 0, 'oduc': 0, 'hop': 0, 'p.ex': 0, 'pag': 0, 'age': 0, 'ew

## Prediction Phase

The algorithm uses pre-computed (trained) feature sets and probabilities to classify a new instance (URL). It involves:

1. Extracting n-grams from the URL.
2. Using these n-grams to calculate genre probabilities based on pre-learned frequencies and interpolation coefficients.
3. Applying backoff to handle unseen n-grams by considering shorter n-grams.
4. Normalizing the resulting probabilities and selecting the most probable genre.

In [4]:
import numpy as np

In [5]:
################################# Normalize the probabilities #################################

def normalize_probs(probs):
    """Normalizes the probability dictionary so that the probabilities sum to 1."""
    total = sum(probs.values())
    if total == 0:
        return {k: 0 for k in probs}  # Return zero probabilities if total is zero
    return {k: v / total for k, v in probs.items()}

################################# Calculate the probability of an n-gram for a given genre #################################

def calculate_ngram_prob(ngram, genre, ngram_freq, genre_counts, lambdas):
    """
    Calculates the probability of an n-gram given a genre using pre-trained frequencies and interpolation coefficients.
    """
    n = len(ngram)
    lambda_n = lambdas.get(n, 0)
    freq = ngram_freq[genre].get(ngram, 0)
    total_ngrams = sum(ngram_freq[genre].values())
    if total_ngrams == 0:
        return 1e-10  # Return a small non-zero value if no n-grams are found
    prob = (lambda_n * (freq / total_ngrams))
    return prob

################################# Extract n-grams from a URL #################################

def extract_ngrams(url, n):
    """Extracts n-grams of length n from the given URL."""
    return [url[i:i+n] for i in range(len(url) - n + 1)]

################################# Prediction function #################################

def predict_genre(url, ngram_freq, genre_counts, lambdas, n_range=(2, 3, 4)):
    """
    Predicts the genre of a given URL using pre-trained n-gram frequencies, genre counts, and interpolation coefficients.
    """
    probs = {genre: np.log(count / sum(genre_counts.values())) if count > 0 else -np.inf for genre, count in genre_counts.items()}
    Q = []
    grams = set()
    
    for n in n_range:
        Q.extend(extract_ngrams(url, n))
    
    while Q:
        gram = Q.pop(0)
        if any(gram in ngram_freq[genre] for genre in genre_counts):
            grams.add(gram)
        else:
            if len(gram) > min(n_range):
                Q.append(gram[:-1])
    
    if grams:
        for genre in genre_counts:
            for gram in grams:
                ngram_prob = calculate_ngram_prob(gram, genre, ngram_freq, genre_counts, lambdas)
                if ngram_prob > 0:
                    probs[genre] += np.log(ngram_prob)
    
    probs = normalize_probs(probs)
    predicted_genre = max(probs, key=probs.get)
    
    return predicted_genre, probs


**Make Precitions:**

In [11]:
urls = [
        
    # Cannabis-related URLs
    "bundesgesundheitsministerium.de/themen/cannabis/faq-cannabisgesetz",
    "de.wikipedia.org/wiki/Cannabis_als_Medizin",
    "cannabis-med.org/index.php?tpl=cannabis",
    
    # Children-related URLs
    "paritaet-bw.de/was-ist-kindergrundsicherung",
    "kindergesundheit-info.de/themen/entwicklung/alle-entwicklungsbereiche/",
    "de.wikipedia.org/wiki/Kindergesundheit",
    
    # Energy-related URLs
    "https://de.wikipedia.org/wiki/Erneuerbare-Energien-Gesetz",
    "bmu.de/themen/energie/erneuerbare-energien",
    "energiewende.de",
    
    # Other URLs
    "https://en.wikipedia.org/wiki/Richard_Feynman",
    "example.com",
    "other-example.org"
]

genres = ["cannabis", "cannabis", "cannabis", "children", "children", "children", "energy", "energy", "energy", "other", "other", "other"]
n_range = (2, 3, 4)

# Train the model
ngram_freq, genre_counts, lambdas = train_genre_classifier(urls, genres, n_range)


In [12]:
# Predict the genre of a new URL
new_url = "tagesschau.de/wirtschaft/cannabis-legalisierung-volkswirtschaft-kosten-100.html"
predicted_genre, probabilities = predict_genre(new_url, ngram_freq, genre_counts, lambdas, n_range)

print("Predicted Genre:", predicted_genre)
print("Genre Probabilities:", probabilities)

Predicted Genre: cannabis
Genre Probabilities: {'cannabis': 0.3968993080538406, 'children': 0.32905631661106044, 'energy': 0.17351466636779836, 'other': 0.1005297089673007}


In [13]:
# Predict the genre of a new URL
new_url = "bundesfinanzministerium.de/Content/DE/Glossareintraege/E/energie-umlage.html"
predicted_genre, probabilities = predict_genre(new_url, ngram_freq, genre_counts, lambdas, n_range)

print("Predicted Genre:", predicted_genre)
print("Genre Probabilities:", probabilities)

Predicted Genre: cannabis
Genre Probabilities: {'cannabis': 0.4574776708384911, 'children': 0.24060528606842502, 'energy': 0.23501095435987104, 'other': 0.06690608873321281}


In [14]:
# Predict the genre of a new URL
new_url = "energy-knowledge.de/Content/DE/Glossareintraege/E/energie-umlage.html"
predicted_genre, probabilities = predict_genre(new_url, ngram_freq, genre_counts, lambdas, n_range)

print("Predicted Genre:", predicted_genre)
print("Genre Probabilities:", probabilities)

Predicted Genre: energy
Genre Probabilities: {'cannabis': 0.21173155555456805, 'children': 0.30906088718925107, 'energy': 0.3647854729017985, 'other': 0.11442208435438236}
